##### Download dataset and split into a training set and a testing set.

In [1]:

##Prepare the dataset
from six.moves import urllib

print("Could not download MNIST data from mldata.org, trying alternative...")

# Alternative method to load MNIST, if mldata.org is down
from scipy.io import loadmat
mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
mnist_path = "./mnist-original.mat"
response = urllib.request.urlopen(mnist_alternative_url)
with open(mnist_path, "wb") as f:
    content = response.read()
    f.write(content)
mnist_raw = loadmat(mnist_path)
mnist = {
    "data": mnist_raw["data"].T,
    "target": mnist_raw["label"][0],
    "COL_NAMES": ["label", "data"],
    "DESCR": "mldata.org dataset: mnist-original",
}
print("Success!")

X,y = mnist['data'], mnist['target']

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


Could not download MNIST data from mldata.org, trying alternative...
Success!


## PART ONE: MINIST classification using scikit learn
###### Set up environment:

In [2]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

# Load scikit's support vector machine library
from sklearn.svm import SVC

# Load pandas
import pandas as pd

# Load numpy
import numpy as np

# Set random seed
np.random.seed(0)

# Load scikit's grid search cross validation library
from sklearn.model_selection import GridSearchCV

# Import accuracy score library
from sklearn.metrics import accuracy_score

##### Perform a 3-fold grid search and train the Random Forest Classifier

In [3]:
# Set grid search parameters
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [100, 300, 784]}]
rfc = RandomForestClassifier()

# Train rfc model across 3 folds
grid_search = GridSearchCV(rfc, param_grid, cv = 3)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [100, 300, 784]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [4]:
# Review best model
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=100, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
# Test final model and calculate accuracy
final_model = grid_search.best_estimator_
y_pred = final_model.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.9654


##### Perform a 3-fold grid search and train the Support Vector Machine Classifier

In [11]:
# Normalize the training data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
std_X_train = scaler.transform(X_train)
std_X_test = scaler.transform(X_test)


/home/jding/anaconda3/envs/myenv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jding/anaconda3/envs/myenv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jding/anaconda3/envs/myenv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
# Set grid parameters
param_grid2 = [
        {'kernel': ['linear'], 'C': [10., 30., 100.],'probability':[False]},
        {'kernel': ['poly'], 'C': [1.0, 3.0, 5.0],'gamma': [0.01, 0.03, 0.08],'probability':[False]},
        {'kernel': ['rbf'],'C': [2.8],'gamma': [0.0073],'probability': [False]}
]
svm = SVC()

# Train svc model across 3 folds using the first 10000 datapoints
grid_search = GridSearchCV(svm, param_grid2, cv = 3)
grid_search.fit(std_X_train[:10000], y_train[:10000])

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'kernel': ['linear'], 'C': [10.0, 30.0, 100.0], 'probability': [False]}, {'kernel': ['poly'], 'C': [1.0, 3.0, 5.0], 'gamma': [0.01, 0.03, 0.08], 'probability': [False]}, {'kernel': ['rbf'], 'C': [2.8], 'gamma': [0.0073], 'probability': [False]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
# Review the best estimators from grid search
grid_search.best_estimator_

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
# Train selected model with the remaining 50000 data points.
svm_trained = SVC(C=1.0, gamma = 0.01, kernel='poly', probability=False)
svm_trained.fit(std_X_train[10000:], y_train[10000:])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
# Test trained model and calculate accuracy
#y_pred2 = svm_trained.predict(X_test)
print("Accuracy: ", svm_trained.score(std_X_test, y_test))

Accuracy:  0.8811
